### TO DO:

- correct the path for the pipeline
- correct the pathes


In [1]:
import os
import time
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage.transform import resize
from skimage import data, exposure
from PIL import Image
from sklearn.svm import SVC #SVM classifier
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
def normalize_and_flatten(features):
    features = features - np.min(features, axis=0, keepdims=True)
    features = features / np.max(features, axis=0, keepdims=True)
    return features.flatten()


In [3]:
resize_factor = (188, 250)

train_img_path = '../data/train_set/train_images/'
test_img_path = '../data/train_set/test_images/'
train_features_path = '../data/train_set/train_points/'
test_features_path = '../data/train_set/test_points/'
labels = pd.read_csv('../data/train_set/label.csv')

train_img = sorted(os.listdir(train_img_path))
test_img = sorted(os.listdir(test_img_path))

## TRUE Labels

In [4]:
### Train emotion indexes
train_indexes = []
for img in train_img:
    if img == ".DS_Store":
        continue
    img = int(img[:4])
    train_indexes.append(img - 1)
    
emotion_idx = labels[['emotion_idx']].loc[train_indexes].values[:, 0]

### Test emotion indexes
test_indexes = []
for img in test_img:
    if img == ".DS_Store":
        continue
    img = int(img[:4])
    test_indexes.append(img - 1)
    
test_emotion_idx = labels[['emotion_idx']].loc[test_indexes].values[:, 0]   

## FEATURES
#### HOGS, train_features, concat_train_features

In [5]:
### extract HOG for train data set

HOGs = []
start_time = time.time()
for img in train_img:
    if img == '.DS_Store':
        continue
    img = np.asarray(Image.open(os.path.join(train_img_path, img)))
    img = resize(img, resize_factor, anti_aliasing=False)
    fetures_hog = hog(img, orientations=9, pixels_per_cell=(8, 8),
                    cells_per_block=(3, 3), visualize=False, multichannel=True)
    HOGs.append(fetures_hog)
print("--- %s seconds ---" % (time.time() - start_time))  

### extract HOG for test data set

HOGs_test = []
start_time = time.time()
for img in test_img:
    if img == '.DS_Store':
        continue
    img = np.asarray(Image.open(os.path.join('../data/train_set/test_images/', img)))
    img = resize(img, resize_factor, anti_aliasing=False)
    fetures_hog = hog(img, orientations=9, pixels_per_cell=(8, 8),
                    cells_per_block=(3, 3), visualize=False, multichannel=True)
    HOGs_test.append(fetures_hog)
print("--- %s seconds ---" % (time.time() - start_time)) 


--- 122.83936095237732 seconds ---
--- 30.634644985198975 seconds ---


In [6]:
### Point features for train dataset (NORMALIZED VERSION)
train_features = sorted(os.listdir(train_features_path))
train_features_list = []
for f in train_features:
    tmp = sio.loadmat(os.path.join(train_features_path, f))
    if 'faceCoordinatesUnwarped' in tmp:
        features = tmp['faceCoordinatesUnwarped']
    else:
        features = tmp['faceCoordinates2']
    train_features_list.append(normalize_and_flatten(features))
    
### Point features for test dataset (NORMALIZED VERSION)
test_features = sorted(os.listdir(test_features_path))
test_features_list = []
for f in test_features:
    tmp = sio.loadmat(os.path.join(test_features_path, f))
    if 'faceCoordinatesUnwarped' in tmp:
        features = tmp['faceCoordinatesUnwarped']
    else:
        features = tmp['faceCoordinates2']
    test_features_list.append(normalize_and_flatten(features))    

In [7]:
### Cancate of HOG and .mat features train
start_time = time.time()
pca = PCA(n_components=100)
reduced_hogs = pca.fit_transform(np.array(HOGs)) 
train_np = np.array(train_features_list)
concat_train_features = np.concatenate((reduced_hogs, train_np), axis=1)
print("--- %s seconds ---" % (time.time() - start_time))

### test
start_time = time.time()
reduced_hogs_test = pca.transform(np.array(HOGs_test))
test_np = np.array(test_features_list)
concat_test_features = np.concatenate((reduced_hogs_test, test_np), axis=1)
print("--- %s seconds ---" % (time.time() - start_time))

--- 14.114684820175171 seconds ---
--- 1.072826862335205 seconds ---


## CLASSIFIERS

#### Linear SVM on HOG

In [8]:
start_time = time.time()
linear_SVM = SVC(kernel="linear", C=0.001)
linear_SVM.fit(np.array(HOGs), emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = linear_SVM.score(np.array(HOGs), emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = linear_SVM.score(np.array(HOGs_test), test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 483.01157999038696 seconds
Train Accuracy Durtion: 259.4689209461212 seconds
Train Accuracy: 0.09404702351175588
Test Accuracy Duration: 68.76027393341064 seconds
Test Accuracy: 0.06786427145708583


#### Linear SVM on .mat features

In [26]:
start_time = time.time()
linear_SVM = SVC(kernel="linear", C=0.001)
linear_SVM.fit(np.array(train_features_list), emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = linear_SVM.score(np.array(train_features_list), emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = linear_SVM.score(np.array(test_features_list), test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 1.283754825592041 seconds
Train Accuracy Durtion: 0.7464189529418945 seconds
Train Accuracy: 0.05202601300650325
Test Accuracy Duration: 0.19126558303833008 seconds
Test Accuracy: 0.06187624750499002


#### L SVM on concat features

In [24]:
start_time = time.time()
linear_SVM = SVC(kernel="linear", C=0.001)
linear_SVM.fit(concat_train_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = linear_SVM.score(concat_train_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = linear_SVM.score(concat_test_features, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 2.1608119010925293 seconds
Train Accuracy Durtion: 1.254235029220581 seconds
Train Accuracy: 0.07253626813406704
Test Accuracy Duration: 0.31401801109313965 seconds
Test Accuracy: 0.0658682634730539


#### L SVM on PCA HOG

In [19]:
start_time = time.time()
pca = PCA(n_components=200)
reduced_features = pca.fit_transform(np.array(HOGs))
linear_SVM = SVC(kernel="linear", C=0.01)
linear_SVM.fit(reduced_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = linear_SVM.score(reduced_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
reduced_test = pca.transform(np.array(HOGs_test))
accuracy = linear_SVM.score(reduced_test, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 16.768794059753418 seconds
Train Accuracy Durtion: 1.0205650329589844 seconds
Train Accuracy: 0.39919959979989994
Test Accuracy Duration: 0.7323682308197021 seconds
Test Accuracy: 0.09580838323353294


#### SVM on PCA HOG

In [38]:
start_time = time.time()
pca = PCA(n_components=100)
reduced_features = pca.fit_transform(np.array(HOGs))
model = SVC(gamma='auto', C=10)
model.fit(reduced_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(reduced_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
reduced_test = pca.transform(np.array(HOGs_test))
accuracy = model.score(reduced_test, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 15.348134279251099 seconds
Train Accuracy Durtion: 0.5401968955993652 seconds
Train Accuracy: 0.9934967483741871
Test Accuracy Duration: 0.5810391902923584 seconds
Test Accuracy: 0.04590818363273453


#### SVM on .mat Features

In [30]:
start_time = time.time()
model = SVC(gamma='auto', C=1000)
model.fit(np.array(train_features_list), emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(np.array(train_features_list), emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = model.score(np.array(test_features_list), test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 0.7238380908966064 seconds
Train Accuracy Durtion: 0.8284599781036377 seconds
Train Accuracy: 0.6183091545772886
Test Accuracy Duration: 0.1888411045074463 seconds
Test Accuracy: 0.46506986027944114


#### SVM on concat features

In [37]:
start_time = time.time()
model = SVC(gamma='auto', C=10)
model.fit(concat_train_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(concat_train_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = model.score(concat_test_features, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 2.657891035079956 seconds
Train Accuracy Durtion: 1.3531501293182373 seconds
Train Accuracy: 0.8539269634817409
Test Accuracy Duration: 0.36635494232177734 seconds
Test Accuracy: 0.07584830339321358


#### RandomForestClassifier on PCA HOG

In [42]:
start_time = time.time()
pca = PCA(n_components=100)
reduced_features = pca.fit_transform(np.array(HOGs))
model = RandomForestClassifier(max_depth=5, n_estimators=50)
model.fit(reduced_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(reduced_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
reduced_test = pca.transform(np.array(HOGs_test))
accuracy = model.score(reduced_test, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 11.663228273391724 seconds
Train Accuracy Durtion: 0.02125692367553711 seconds
Train Accuracy: 0.5317658829414708
Test Accuracy Duration: 0.337507963180542 seconds
Test Accuracy: 0.06187624750499002


#### RFC on .mat features

In [47]:
start_time = time.time()
model = RandomForestClassifier(max_depth=5, n_estimators=100)
model.fit(np.array(train_features_list), emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(np.array(train_features_list), emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = model.score(np.array(test_features_list), test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 1.1041409969329834 seconds
Train Accuracy Durtion: 0.030183076858520508 seconds
Train Accuracy: 0.5157578789394698
Test Accuracy Duration: 0.011514902114868164 seconds
Test Accuracy: 0.3013972055888224


#### RFC on cancate features

In [48]:
start_time = time.time()
model = RandomForestClassifier(max_depth=5, n_estimators=100)
model.fit(concat_train_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(concat_train_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = model.score(concat_test_features, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 1.3742477893829346 seconds
Train Accuracy Durtion: 0.05086398124694824 seconds
Train Accuracy: 0.5327663831915957
Test Accuracy Duration: 0.025084972381591797 seconds
Test Accuracy: 0.3073852295409182


#### MLPClassifier on pca HOG

In [49]:
start_time = time.time()
pca = PCA(n_components=100)
reduced_features = pca.fit_transform(np.array(HOGs))
model = MLPClassifier(hidden_layer_sizes=(300, 600, 300, 150, 60, 40))
model.fit(reduced_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(reduced_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
reduced_test = pca.transform(np.array(HOGs_test))
accuracy = model.score(reduced_test, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 19.290612936019897 seconds
Train Accuracy Durtion: 0.04608917236328125 seconds
Train Accuracy: 0.9904952476238119
Test Accuracy Duration: 0.9957513809204102 seconds
Test Accuracy: 0.08183632734530938


#### MLP on .mat

In [50]:
start_time = time.time()
model = MLPClassifier(hidden_layer_sizes=(300, 600, 300, 150, 60, 40))
model.fit(np.array(train_features_list), emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(np.array(train_features_list), emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = model.score(np.array(test_features_list), test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 3.587941884994507 seconds
Train Accuracy Durtion: 0.0499267578125 seconds
Train Accuracy: 0.1265632816408204
Test Accuracy Duration: 0.011600971221923828 seconds
Test Accuracy: 0.10778443113772455


#### MLP on concat features

In [52]:
start_time = time.time()
model = MLPClassifier(hidden_layer_sizes=(300, 600, 300, 60, 40))
model.fit(concat_train_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(concat_train_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = model.score(concat_test_features, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 5.981200933456421 seconds
Train Accuracy Durtion: 0.04111790657043457 seconds
Train Accuracy: 0.9959979989994997
Test Accuracy Duration: 0.010310888290405273 seconds
Test Accuracy: 0.11976047904191617
